In [1]:
import requests

from address import Address # python wrapper for ERGO address class
from base64 import b64encode

### Config
_walletAddress_ - ERGO wallet that initiates and pays the smart contract<br>
_toAddress_ - ERGO wallet that will recieve result of ergoscript (if there is one)<br>

In [2]:
# testnet
walletAddress = '3WwjaerfwDqYvFwvPRVJBJx2iUvCjD2jVpsL82Zho1aaV5R95jsG'
api_key       = 'oncejournalstrangeweather' # setup for node; assembler uses this

In [ ]:
# mainnet
walletAddress = '9iD7JfYYemJgVz7nTGg9gaHuWg7hBbHo2kxrrJawyz4BD1r9fLS'
api_key       = 'fancyapikeygoeshere' # setup for node; assembler uses this

In [50]:
# ergonode api routes
HOST = 'localhost'
PORT = 9053
ergonode = f'http://{HOST}:{PORT}'
wallet_transaction_generateUnsigned = f'{ergonode}/wallet/transaction/generateUnsigned'
wallet_transaction_sign = f'{ergonode}/wallet/transaction/sign'
wallet_transaction_send = f'{ergonode}/wallet/transaction/send'
ergo_info = f'{ergonode}/info'

headers       = {'Content-Type': 'application/json'}

# wallet to expect funding from
userWallet    = Address(walletAddress)
userTreeHex   = userWallet.ergoTree()
userTree      = b64encode(bytes.fromhex(userTreeHex)).decode()

In [5]:
minTx          = 10000000
txFee          = 2000000
nanoergsInErg  = 1000000000     # 1e9
ergAmount      = .1             # TODO: how much to send in transaction
qtyTokens      = 10000          # TODO: number of tokens
decimals       = 0              # TODO: (optional) can change this
name           = 'ASDFToken123' # TODO: change token name
description    = 'ASDFToken'    # TODO: change token description
nergAmount     = int(ergAmount * nanoergsInErg)

In [58]:
def mintToken(amount, name, description, decimals):
    ergAmount      = .1             # TODO: how much to send in transaction?
    nanoergsInErg  = 1000000000     # 1e9
    nergAmount     = int(ergAmount * nanoergsInErg)
    minTx          = 10000000
    creationHeight = 0

    res = requests.get(ergo_info)
    if res.ok:
        creationHeight = res.json()['fullHeight']

    # outBox
    req = {
        "requests": [
            {
                'address': userWallet.address,
                'ergValue': nergAmount + minTx,
                'amount': amount,
                'name': name,
                'description': description,
                'decimals': decimals, # 0-9
            },
        ],
        'inputs': [],
        'dataInputs': [],
        'outputs': []
    }

    # generate unsigned transaction with token info
    res = requests.post(wallet_transaction_generateUnsigned, headers=dict(headers, **{'api_key': api_key}), json=req)
    
    # return the unsigned transaction
    if res.ok:
        return ({ 'tx': res.json() }, req)
    else:
        return ({}, {})

In [59]:
tx, req = mintToken(101, 'hello101', 'world token', 8)
req

{'requests': [{'address': '3WwjaerfwDqYvFwvPRVJBJx2iUvCjD2jVpsL82Zho1aaV5R95jsG',
   'ergValue': 110000000,
   'amount': 101,
   'name': 'hello101',
   'description': 'world token',
   'decimals': 8}],
 'inputs': [],
 'dataInputs': [],
 'outputs': []}

In [60]:
res = requests.post(wallet_transaction_send, headers=dict(headers, **{'api_key': api_key}), json=req)
res.json()

b'"aac6698a7dbf0086e8a6c80d70481e8ed1ffb95c555fc94fb3c89588d847d881"'

In [57]:
req

{'tx': {'id': '7c58c1b4cafa08767a939b5f25e1f929fff6beb914871f7ae413e54e3b702900',
  'inputs': [{'boxId': '02b5589cfb2df5b1af147378ef80585562451306779dcdbd97fb5ba9a88ccd6b',
    'extension': {}},
   {'boxId': 'f93ca82a52e708ee97f541ef15d55b0ce170a878620cf76422a4d92519fee83e',
    'extension': {}},
   {'boxId': '26e0f31458531ba52e38610b65d0d892ddac1a0c94d1cb673943aad657ab2a7b',
    'extension': {}}],
  'dataInputs': [],
  'outputs': [{'boxId': 'ba76aaa1948482a6900af2f5ba4b6cb82cc66d6a8228e8963777d463774a0153',
    'value': 110000000,
    'ergoTree': '0008cd02946f31c13a75cb07571a63e860fa79ebd866278651cb1268d0c70c8c9beaaf4e',
    'assets': [{'tokenId': '02b5589cfb2df5b1af147378ef80585562451306779dcdbd97fb5ba9a88ccd6b',
      'amount': 101}],
    'creationHeight': 114838,
    'additionalRegisters': {'R4': '0e0868656c6c6f313031',
     'R5': '0e0b776f726c6420746f6b656e',
     'R6': '0e0138'},
    'transactionId': '7c58c1b4cafa08767a939b5f25e1f929fff6beb914871f7ae413e54e3b702900',
    'index':

In [41]:
# make payment to smart contract
# req = mintToken(100, 'hello100', 'world token', 8)
res = requests.post(wallet_transaction_sign, headers=dict(headers, **{'api_key': api_key}), json=req)

In [43]:
request = ''
if res.ok:
    request = res.json()
    print(res.json())
else:
    print(f'ERR: status code {res.status_code}\n{res.content}')

{'id': 'acfc88b23d04276c5f3ffbd1b759f3c0546c550b72eff49469808aea4280609e', 'inputs': [{'boxId': '02b5589cfb2df5b1af147378ef80585562451306779dcdbd97fb5ba9a88ccd6b', 'spendingProof': {'proofBytes': '969dd8b8da9d0fcb1815b4311e2b824ca27e88f5f3dae71ee6fad8d4fb5c39da06c10d2b660e8a03f46afb6db00d55fce607188bc574b968', 'extension': {}}}, {'boxId': 'f93ca82a52e708ee97f541ef15d55b0ce170a878620cf76422a4d92519fee83e', 'spendingProof': {'proofBytes': '8879f027ffb286dce87a643e2fe0118826c7ef52fa074865318a9ffd6d4fa56a9f391ed35951cfba618db72db774adecf424349c9e749a3b', 'extension': {}}}, {'boxId': '26e0f31458531ba52e38610b65d0d892ddac1a0c94d1cb673943aad657ab2a7b', 'spendingProof': {'proofBytes': '4e5a03a1661a625b3f3947b88f099558ad4cc93fda71d96342e029adcf33708e88194231f78e3b655b4305ab3bec8c750f6a43ed79c2182a', 'extension': {}}}], 'dataInputs': [], 'outputs': [{'boxId': '5e62e405395e2837670a076f271205d4e1d697286f133b1feb9e5bb71be79684', 'value': 110000000, 'ergoTree': '0008cd02946f31c13a75cb07571a63e860fa7

In [48]:
assembler_url = 'http://localhost:8080'
res = requests.post(f'{assembler_url}/follow', headers=headers, json=request)
res.content
# id = res.json()['id']
# id

b'{\n  "success": false,\n  "detail": "address field is required"\n}'